In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import numpy as np
import glob
import h5py
from tqdm.auto import tqdm
from markovids import vid
from joblib import Parallel, delayed

## User functions

In [4]:
def get_rolling_bground(
    dat_file,
    step_size=1500,
    frame_spacing=20,
    overlap=0,
    agg_func=np.mean,
    reader_kwargs={"threads": 2},
    progress_bar=False,
    save_dir="_bground",
):

    basename = os.path.splitext(os.path.basename(dat_file))[0]
    path = os.path.dirname(dat_file)
    bground_path = os.path.join(path, save_dir, f"{basename}.hdf5")

    os.makedirs(os.path.join(path, save_dir), exist_ok=True)

    if os.path.exists(bground_path):
        return None

    vid_reader = vid.io.AutoReader(dat_file, **reader_kwargs)
    steps = range(0, vid_reader.nframes, step_size - overlap)

    bgrounds = []
    idxs = []
    for _step in tqdm(steps, disable=not progress_bar):
        left_edge = _step
        right_edge = left_edge + step_size

        if right_edge > vid_reader.nframes:
            break
        # right_edge = min(left_edge + step_size, reader.nframes)
        use_frames = vid_reader.get_frames(range(left_edge, right_edge, frame_spacing))
        bgrounds.append(agg_func(use_frames, axis=0))
        idxs.append((left_edge + right_edge) / 2)  # get midpoint frame index...
    vid_reader.close()

    with h5py.File(bground_path, "w") as f:
        f.create_dataset(
            "bground",
            data=np.array(bgrounds),
            compression="gzip",
        )
        f.create_dataset("frame_idxs", data=np.array(idxs), compression="gzip")

    return None

# Quantify fluorescence decay over time

In [ ]:
base_dir = os.path.expanduser("~/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/timecourse_0*")
fluo_files = sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))

In [8]:
base_dir = os.path.expanduser("~/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/sciadv_rebuttal/")
fluo_files += sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))

In [9]:
delays = []
for _file in fluo_files:
    delays.append(
        delayed(get_rolling_bground)(
            _file,
        )
    )
print(len(delays))
dat = Parallel(n_jobs=8, verbose=10, batch_size=8, backend="multiprocessing")(delays)

1832


[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done  38 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  88 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 144 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 216 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done 288 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done 376 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done 464 tasks      | elapsed:    3.2s
[Parallel(n_jobs=8)]: Done 568 tasks      | elapsed:    3.7s
[Parallel(n_jobs=8)]: Done 672 tasks      | elapsed:    4.3s
[Parallel(n_jobs=8)]: Done 792 tasks      | elapsed:    5.5s
[Parallel(n_jobs=8)]: Done 912 tasks      | elapsed:    6.6s
[Parallel(n_jobs=8)]: Done 1048 tasks      | elapsed:    7.2s
[Parallel(n_jobs=8)]: Done 1184 tasks      | elapsed:    8.2s
[Parallel(n_jobs=8)]: Done 1336 tasks      | elapsed:    9